# Warehouse Problem - Lagrangian relaxation

Let us recall the original MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c}  & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

In [ ]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 2 are relaxed.

#### 2. Implement the lagrangian relaxation.

In [ ]:
import gurobipy as gp

def lagrangian_relaxation(lagrange_multipliers=[1] * nbStores):
    mdl = gp.Model(name='lagrangian')

    # Variables
    warehouse_is_open = mdl.addVars(nbWarehouses, vtype=gp.GRB.BINARY, name='warehouse_is_open')
    warehouse_supplies_store = mdl.addVars(Warehouses, Stores, vtype=gp.GRB.BINARY, name='warehouse_supplies_store')

    # Constraint
    only_open_warehouses_can_serve_stores = mdl.addConstrs(warehouse_supplies_store[w, c] <= warehouse_is_open[w] for w in Warehouses for c in Stores)
    # This is the constraint that we're going to relax.
    # each_store_is_served_by_one_warehouse = mdl.addConstrs(gp.quicksum(warehouse_supplies_store[w, c] for w in Warehouses) == 1 for c in Stores)

    warehouse_capacity_is_respected = mdl.addConstrs(gp.guicksum(warehouse_supplies_store[w, c] for c in Stores) <= capacity[w]*warehouse_is_open[w] for w in Warehouses)


    # Objective function
    cost_of_opening_a_warehouse = gp.quicksum(fixed*warehouse_is_open[w] for w in Warehouses)
    transportation_cost = gp.quicksum(supplyCost[w][c]*warehouse_supplies_store[w, c] for w in Warehouses for c in Stores)
    lagrange_cost = gp.quicksum(1 - lagrange_multipliers[c] * gp.quicksum(warehouse_supplies_store[w, c] for w in Warehouses) for c in Stores)

    mdl.setObjective(cost_of_opening_a_warehouse + transportation_cost + lagrange_cost, sense=gp.GRB.MINIMIZE)

    mdl.optimize()

    msol = (
        [var.X for var in warehouse_is_open.values()],
        [var.X for var in warehouse_supplies_store.values()],
    )

    relaxed_constraints = [1 - gp.quicksum(msol[1][w][c] for w in Warehouses) for c in Stores]
    
    return (mdl.getObjective().getValue(),  # the objective value
            msol, # solution
            relaxed_constraints)  # slack of the relaxed constraints

#### Try the function

In [ ]:
from pprint import pprint

obj, msol, relaxed = lagrangian_relaxation([fixed]*nbStores)
pprint(msol)
pprint(relaxed)

#### 3. Implement the Cutting Plane Method

1. State the master problem
2. Keep track of the best feasible solution
3. Use the solution to the master problem to solve the Lagrangian subproblem

In [ ]:
rlambda = [fixed]*nbStores
fsol = None
iteration = 0
step = 1
UB = M*nbStores
bestFeasibleValue = UB
bestFeasible = 0

while True:
    # TODO: implement the loop
    
  
print('========================')
print('Lagrangian dual: ' + str(UB))
if bestFeasible != 0:
    print('-- best feasible --')
    print(bestFeasible)